# File: dataset_processing.py

In [5]:
# The autoreload extension allows you to tweak the code in the imported modules
# and rerun cells to reflect the changes.
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## Managing Data

In this section we demonstrate the implemented class that helps you to manage training data and results.

In [7]:
from dataset_processing import *

One file can only be accessed by one class at a time:

In [ ]:
shhs_data_manager = DataManager(file_path = "messing_around.pkl")
shhs_data_manager = DataManager(file_path = "messing_around.pkl")

## Introduction to the implemented functions

In [8]:
import numpy as np # type: ignore
import random
import h5py # type: ignore

In this section you can check whether the implemented functions in this project work correctly.

### Scaling number of datapoints from signal- to target- frequency:

I would highly suggest to provide data where the signals don't need to be scaled to the frequencies of the data
used to train the neural network.

If there is no other option, then so be it. Here is a demonstration of the functions that will be applied to 
your data:

#### Classification Signal

In [4]:
classification_array = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
classification_frequency = 1/20
target_frequency = 1/30

print("-"*71)
print(f"Classification Frequency: {classification_frequency} -> Target Frequency: {target_frequency}")
print("-"*71)
print("\nClassification array: ", classification_array)
print("Classification array shape: ", classification_array.shape)

reshaped_array = scale_classification_signal(
        signal = classification_array, 
        signal_frequency = classification_frequency,
        target_frequency = target_frequency
        )

print("\nScaled array: ", reshaped_array)
print("Scaled array shape: ", reshaped_array.shape)

classification_array = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
classification_frequency = 1/50
target_frequency = 1/30

print("\n")
print("-"*71)
print(f"Classification Frequency: {classification_frequency} -> Target Frequency: {target_frequency}")
print("-"*71)
print("\nClassification array: ", classification_array)
print("Classification array shape: ", classification_array.shape)

reshaped_array = scale_classification_signal(
        signal = classification_array, 
        signal_frequency = classification_frequency,
        target_frequency = target_frequency
        )

print("\nScaled array: ", reshaped_array)
print("Scaled array shape: ", reshaped_array.shape)

del reshaped_array, classification_array, classification_frequency, target_frequency

-----------------------------------------------------------------------
Classification Frequency: 0.05 -> Target Frequency: 0.03333333333333333
-----------------------------------------------------------------------

Classification array:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
Classification array shape:  (15,)

Scaled array:  [ 0  1  3  4  6  7  9 10 12 13]
Scaled array shape:  (10,)


-----------------------------------------------------------------------
Classification Frequency: 0.02 -> Target Frequency: 0.03333333333333333
-----------------------------------------------------------------------

Classification array:  [0 1 2 3 4 5 6 7 8]
Classification array shape:  (9,)

Scaled array:  [0 1 1 2 2 3 4 4 5 5 6 7 7 8 8]
Scaled array shape:  (15,)


#### Continuous Signal

In [5]:
continuous_array_int = np.array([0, 1, 2, 3, 4, 5])
continuous_array_float = np.array([0, 1, 2, 3, 4, 5], dtype = float)
continuous_frequency = 3
target_frequency = 4

print("-"*75)
print(f"Continuous Frequency: {continuous_frequency} -> Target Frequency: {target_frequency}")
print("-"*75)
print(f"Continuous array: {continuous_array_int} / {continuous_array_float}")
print("Continuous array shape: ", continuous_array_int.shape)

reshaped_array_int = interpolate_signal(
        signal = continuous_array_int, 
        signal_frequency = continuous_frequency,
        target_frequency = target_frequency
        )

reshaped_array_float = interpolate_signal(
        signal = continuous_array_float, 
        signal_frequency = continuous_frequency,
        target_frequency = target_frequency
        )

print(f"\nScaled array: {reshaped_array_int} / {reshaped_array_float}")
print("Scaled array shape: ", reshaped_array_int.shape)

print("\n")

continuous_array_int = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
continuous_array_float = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype = float)
continuous_frequency = 5
target_frequency = 4

print("-"*75)
print(f"Continuous Frequency: {continuous_frequency} -> Target Frequency: {target_frequency}")
print("-"*75)
print(f"Continuous array: {continuous_array_int} / {continuous_array_float}")
print("Continuous array shape: ", continuous_array_int.shape)

reshaped_array_int = interpolate_signal(
        signal = continuous_array_int, 
        signal_frequency = continuous_frequency,
        target_frequency = target_frequency
        )

reshaped_array_float = interpolate_signal(
        signal = continuous_array_float, 
        signal_frequency = continuous_frequency,
        target_frequency = target_frequency
        )

print(f"\nScaled array: {reshaped_array_int} / {reshaped_array_float}")
print("Scaled array shape: ", reshaped_array_int.shape)

del reshaped_array_int, reshaped_array_float, continuous_array_int, continuous_array_float, continuous_frequency, target_frequency

---------------------------------------------------------------------------
Continuous Frequency: 3 -> Target Frequency: 4
---------------------------------------------------------------------------
Continuous array: [0 1 2 3 4 5] / [0. 1. 2. 3. 4. 5.]
Continuous array shape:  (6,)

Scaled array: [0 1 2 2 3 4 4 5] / [0.   0.75 1.5  2.25 3.   3.75 4.5  5.  ]
Scaled array shape:  (8,)


---------------------------------------------------------------------------
Continuous Frequency: 5 -> Target Frequency: 4
---------------------------------------------------------------------------
Continuous array: [0 1 2 3 4 5 6 7 8 9] / [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
Continuous array shape:  (10,)

Scaled array: [0 1 2 4 5 6 8 9] / [0.   1.25 2.5  3.75 5.   6.25 7.5  8.75]
Scaled array shape:  (8,)


### Splitting a signal which is too long for the neural network model

A signal which is too short will be padded with zeros. No big deal. On the other hand: A signal which is too 
long will be splitted into multiple signals. To create more data, the 10 hour range will be shifted along the
signal.

This shift should not be too small, to create redundant data but also not too big, because the more data the 
better. So we try to find a shift size close to 1 hour, which lets us shift an integer amount of times
easily. 

#### Finding optimal shift size

In [6]:
signal_length_addition_hours = 2.5
desired_length_hours = 10

optimal_shift_length = calculate_optimal_shift_length(
        signal_length = (desired_length_hours + signal_length_addition_hours) * 3600, # type: ignore
        desired_length = desired_length_hours*3600, 
        wanted_shift_length = 3600,
        absolute_shift_deviation = 1800,
)

print(f"Optimal shift length for signal which is {signal_length_addition_hours} hours longer than desired length of {desired_length_hours} hours: {round(optimal_shift_length/3600, 3)} hours")

Optimal shift length for signal which is 2.5 hours longer than desired length of 10 hours: 0.833 hours


#### Splitting Signal

The above function to find optimal shift length is embedded in the following split funtion. The optimal
shift size will be estimated for every signal individually.

If there is no integer shift size in range, that lets you shift the signal so, that you perfectly enclose the
last datapoints of the long signal, then the last shift will be altered so that it does.

In [7]:
# Create random signal
frequency = 4
length_signal_seconds = 12.1 * 3600
signal = np.random.rand(int(length_signal_seconds * frequency))

# Only important parameters here:
nn_signal_seconds = 10 * 3600
shift_length_seconds = 3600
absolute_shift_deviation_seconds = 1800

signals_from_splitting, shift_length = split_long_signal(
        signal = signal, 
        sampling_frequency = frequency,
        target_frequency = frequency,
        signal_type = "feature",
        nn_signal_duration_seconds = nn_signal_seconds,
        wanted_shift_length_seconds = shift_length_seconds,
        absolute_shift_deviation_seconds = absolute_shift_deviation_seconds
        )

print("Shift length:", shift_length)
print(f"Shift length: {shift_length / frequency} seconds")
print("Signal shape: ", signal.shape)
print(f"Datapoints in NN: {nn_signal_seconds * frequency}")
print("Signals from splitting shape: ", signals_from_splitting.shape)

del signals_from_splitting, signal, shift_length, frequency, nn_signal_seconds, shift_length_seconds, absolute_shift_deviation_seconds

Shift length: 15120
Shift length: 3780.0 seconds
Signal shape:  (174240,)
Datapoints in NN: 144000
Signals from splitting shape:  (3, 144000)


#### Splitting signals within dictionary

In [ ]:
# Create random signal
frequency = 4
length_signal_seconds = 12.1 * 3600
signal = np.random.rand(int(length_signal_seconds * frequency))

# Only important parameters here:
nn_signal_seconds = 10 * 3600
shift_length_seconds = 3600
absolute_shift_deviation_seconds = 1800

### Reading a .h5 - file

One of the available training datasets for our neural network model are stored in a .h5 file. So we need
to be able to read it. These are the important operations:

In [8]:
shhs_dataset = h5py.File("../Training_Data/SHHS_dataset.h5", 'r')
patients = list(shhs_dataset['slp'].keys()) # type: ignore

random_patient = patients[np.random.randint(0, len(patients))]
print(f"Random patient: {random_patient}")

print(np.unique(shhs_dataset["slp"][random_patient][:])) # type: ignore

for key in ["slp", "rri"]:
    print(f"\nkey: {key}")

    patient_data = shhs_dataset[key][random_patient][:] # type: ignore
    print(f"Data shape: {patient_data.shape}") # type: ignore

    data_freq = shhs_dataset[key].attrs["freq"] # type: ignore
    print(f"Data frequency: {data_freq}")
    print(f"Inverse data frequency: {1/data_freq}") # type: ignore

    print(f"Data length: {patient_data.shape[0]/data_freq} s") # type: ignore

del shhs_dataset, patients, random_patient, key, patient_data, data_freq

Random patient: 201558_1
[0 1 2 3 5]

key: slp
Data shape: (1039,)
Data frequency: 0.03333333333333333
Inverse data frequency: 30.0
Data length: 31170.0 s

key: rri
Data shape: (124680,)
Data frequency: 4
Inverse data frequency: 0.25
Data length: 31170.0 s


### Divide up a signal into overlapping windows

The hardest thing about this is, that 'window_overlap' and 'datapoints_per_window' must be chosen so that
the whole signal fits perfectly into n windows. 

Additionally, those values must be integers. This means that 'window_duration_seconds' and 'overlap_seconds'
multiplied with 'target_fequency' as well as 'sampling_frequency' must be integers. (The features and the target labels
must fit equally well into the windows, so that we can find the correlation between a feature- and target- window.)

We have the RRI and MAD values as features and the sleep phase as target classification. As we will see,
RRI and MAD values were recorded with an integer sampling frequency. While the sampling frequency of the 
sleep classification is 1/30. 

Finding window parameters that fullfill the conditions mentioned is easier than it sounds. We will always pass data
to the neural network that is 10 hours long. Now, we just need to think in seconds and find integer values
for 'window_duration_seconds' and 'overlap_seconds' that are a multiple of 30:

#### Finding optimal window_parameters:

In [9]:
find_suitable_window_parameters(
        signal_length = 10 * 3600,
        number_windows_range = (1000, 1400),
        window_size_range = (120, 180),
        minimum_window_size_overlap_difference = 30
    )

Suitable window parameters for signal of length: 36000:
-------------------------------------------------------
Number of windows: 1025, Window size: 160, Overlap: 125.0
Number of windows: 1026, Window size: 125, Overlap: 90.0
Number of windows: 1055, Window size: 164, Overlap: 130.0
Number of windows: 1056, Window size: 130, Overlap: 96.0
Number of windows: 1087, Window size: 162, Overlap: 129.0
Number of windows: 1088, Window size: 129, Overlap: 96.0
Number of windows: 1121, Window size: 160, Overlap: 128.0
Number of windows: 1122, Window size: 128, Overlap: 96.0
Number of windows: 1157, Window size: 164, Overlap: 133.0
Number of windows: 1158, Window size: 133, Overlap: 102.0
Number of windows: 1196, Window size: 150, Overlap: 120.0
Number of windows: 1197, Window size: 120, Overlap: 90.0


Our options are:

Number of windows: 1196, Window size: 150, Overlap: 120.0 \
Number of windows: 1197, Window size: 120, Overlap: 90.0

We will choose the latter, because we don't want the window_size to be too large.

#### Classification Signal

When transforming a classification signal into windows, which is supposed to be the target in the neural 
network, then each window will only be represented by the most common sleep stage. If there is a tie
between the labels, then the one with the highest priority will be chosen 

In [10]:
signal_length_seconds = 10 * 3600
frequency = 1/30
signal_length = int(signal_length_seconds * frequency)

signal = np.array([random.randint(0, 5) for _ in range(signal_length)])

signal_in_windows = signal_to_windows(
    signal = signal, # type: ignore
    datapoints_per_window = int(120 * frequency),
    window_overlap = int(90 * frequency),
    signal_type = "target",
    priority_order=[0, 1, 2, 3, 4, 5]
    )

print(f"Signal shape: {signal.shape}")
print(f"Signal in windows shape: {signal_in_windows.shape}")

del signal, signal_in_windows, signal_length_seconds, frequency, signal_length

Signal shape: (1200,)
Signal in windows shape: (1197,)


#### Continuous Signal

In [11]:
signal = np.random.rand(36000)

signal_in_windows = signal_to_windows(
    signal = signal, # type: ignore
    datapoints_per_window = 120,
    window_overlap = 90,
    signal_type = "feature"
    )

print(f"Signal shape: {signal.shape}")
print(f"Signal in windows shape: {signal_in_windows.shape}")

del signal, signal_in_windows

Signal shape: (36000,)
Signal in windows shape: (1197, 120)


#### Reshape Signal

The following function will actually be applied the transform a signal into windows. It will make sure
that the data is passed correctly to the function mentioned above. 

This means it will:
- correct freuqency of signal if needed
- check if 'number_nn_datapoints', 'datapoints_per_window' and 'window_overlap' are integers
- check if 'datapoints_per_window' and 'window_overlap' perfectly fit into 'number_nn_datapoints'
- compare length of provided signal to length of signal in nn ('number_nn_datapoints')
    - if smaller: Pad with Zeros
    - if bigger: Print warning, but continue by cropping last datapoints
- check if signal transformed to windows has the right shape

In [14]:
random_array = np.random.rand(36000)
reshaped_array = reshape_signal(
    signal = random_array, # type: ignore
    sampling_frequency = 1,
    target_frequency = 4, 
    number_windows = 1197, 
    window_duration_seconds = 120, 
    overlap_seconds = 90,
    signal_type = "feature",
    nn_signal_duration_seconds = 10*3600,
    )

print(f"Random array shape: {random_array.shape}")
print(f"Reshaped array shape: {reshaped_array.shape}")

random_array = np.array([random.randint(0, 5) for _ in range(int(36000/30))])
reshaped_array = reshape_signal(
    signal = random_array, # type: ignore
    sampling_frequency = 1/30,
    target_frequency = 1/30, 
    number_windows = 1197, 
    window_duration_seconds = 120, 
    overlap_seconds = 90,
    signal_type = "target",
    nn_signal_duration_seconds = 10*3600,
    )

print(f"Random array shape: {random_array.shape}")
print(f"Reshaped array shape: {reshaped_array.shape}")

del random_array, reshaped_array

Random array shape: (36000,)
Reshaped array shape: (1197, 480)
Random array shape: (1200,)
Reshaped array shape: (1197,)


change what is padded depending on target and feature